In [8]:
#upload data from csv
import pandas as pd
videosGB = pd.read_csv('OriginalCSVs/GBvideos.csv', sep=',', decimal='.', header=0)
videosUS = pd.read_csv('OriginalCSVs/USvideos.csv', sep=',', decimal='.', header=0)

#concat videos
videos = pd.concat([videosGB,videosUS])

#GB: 19000 videos
#GB+US: 39000 videos
videos.shape

(39549, 16)

In [9]:
#elimina videos duplicados
videos = videos.drop_duplicates('video_id','last',False)
#elimina sem categoria
nodes = videos[videos.tags != '[none]']
        
#cria csv
newheader = ['id','trending_date','Label','channel_title','category_id','publish_time','tags','views','likes','dislikes','comment_count','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
nodes.to_csv('nodes.csv',header=newheader,index=False)

#GB: 2000 nos
#GB+US: 5320 nos
nodes.shape

(5320, 16)

In [10]:
#get tags
nodesTemp = nodes.loc[:, ['video_id','tags']]
sources = []
targets = []

#auxiliar vectors that save the indexes to access to the nodes quickly
sourcesAux = []
targetsAux = []

#criar vetor de tags
for index, tags in nodesTemp.iterrows():
    tags['tags'] = tags['tags'].split('"|"')  
    #compor a ultima tag
    size = len(tags['tags'])
    last = tags['tags'][size-1]
    tags['tags'][size-1] = last.split('"')[0]
    #compor a primeira tag
    tag1 = tags['tags'][0].split('|"')
    tags['tags'][0] = tag1[0]
    if len(tag1) == 2:
        tags['tags'].append(tag1[1])
        
print("done")        

done


In [11]:
def comparelists( id1, list1, index1, id2, list2, index2 ):
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                sources.append(id1)
                sourcesAux.append(index1)
                targets.append(id2)
                targetsAux.append(index2)
                return
         
#para cada node vamos procurar todos aqueles que tem a mesma tag    
for i1, tags1 in nodesTemp.iterrows(): 
    for i2, tags2 in nodesTemp.iterrows(): 
        #compara com os seguintes se existir tags
        if i2 > i1: 
            #verificar se existe alguma tag igual
            comparelists(tags1['video_id'],tags1['tags'],i1,tags2['video_id'],tags2['tags'],i2)

print("done")

done


In [12]:
#create dataframe            
df = {'Source' : sources, 'Target' : targets}   
edges = pd.DataFrame(df)
edges['Type'] = 'Undirected'
edges.to_csv('edges.csv',index=False)

#create auxiliary dataframe
dfAux = {'Source' : sourcesAux, 'Target' : targetsAux}   
edgesAux = pd.DataFrame(dfAux)

#GB: 86000 edges
#GB+US: 574000 edges
edges.shape

(574128, 3)

In [13]:
#criar edges com pesos de acordo com o numero de tags em comum
weights = []
def getweights( ite, list1, list2 ):
    w = 0
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                w += 1
                break
    weights.append(w)

    
#source and target iterator
for e1, edge in edgesAux.iterrows(): 
    list1 = nodesTemp['tags'][edge['Source']]
    list2 = nodesTemp['tags'][edge['Target']]
    getweights(e1, list1, list2)
    
print("done")

done


In [14]:
edgesWeight = edges
edgesWeight['Weight'] = weights
edgesWeight.to_csv('edgesWeightned.csv',index=False)

#remover todos os edges com peso = 1
#GB: 24000 edges
#GB+US: 351498 edges
edgesWeight1 = edgesWeight[edgesWeight.Weight != 1]
edgesWeight1.to_csv('edgesWeightned1.csv',index=False)
edgesWeight1.shape

(351498, 4)